### 从所有的基因数据中选取需要处理的500多个的子集

In [36]:
from openpyxl import Workbook
import os

csv_all = r'D:\项目资料\基因表达\数据\TCGA-COAD.htseq_counts.tsv'
csv_select = r'D:\项目资料\基因表达\数据\Diff_genes_name.csv'
xlsx_out = r'genes.xlsx'
pic_data = r'E:\DX-color-result0702\train'
name_length = 12

# 500多个基因
gene_dict = {}
with open(csv_select) as f:
    for line in f.readlines()[1:]:
        arr = line.strip().split(',')
        gene_dict[arr[1]] = arr[2]

# 输出文件
wb = Workbook()
sheet = wb.create_sheet('Sheet1')

cols = []
gene_data = {}
with open(csv_all) as f:
    for idx, line in enumerate(f.readlines()):
        arr = line.strip().split('\t')
        if idx == 0:
            for name in arr[1:]:
                cols.append(name[:name_length])
        else:
            gene_id = arr[0].split('.')[0]
            if gene_id in gene_dict:
                gene_data[gene_id] = arr[1:]
print('基因字典，筛选出的基因：', len(gene_dict), len(gene_data))
    

# 有数据的目录
folders = []
for folder1 in os.listdir(pic_data):
    folder_path = os.path.join(pic_data, folder1)
    for folder2 in os.listdir(folder_path):
#         print(folder2[:16])
        if folder2[:name_length] in cols:
            folders.append(folder2)
folders = set(folders)

print(len(folders))

    
wb.save(xlsx_out)
wb.close()

基因字典，筛选出的基因： 537 537
306


In [1]:
import numpy as np
import os
import time
from PIL import Image
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import keras
from tensorflow.keras import layers, datasets
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
  tf.config.experimental.set_memory_growth(device, True)

In [4]:
base_model = tf.keras.applications.resnet50.ResNet50(
            include_top=False,
            weights='imagenet',
            input_shape=(224, 224, 3)
        )

In [5]:
train_img_gen = ImageDataGenerator(fill_mode='nearest',
                                   width_shift_range = 0.15,
                                    height_shift_range = 0.15,
                                    shear_range = 0,
                                    rotation_range = 15,
                                    zoom_range = 0.1,
                                    vertical_flip=False,
                                    horizontal_flip=False
                                                 ,samplewise_center=True
                                                 ,samplewise_std_normalization=True
                                                ) 

val_img_gen = ImageDataGenerator(fill_mode='nearest'
                                                 ,samplewise_center=True
                                                 ,samplewise_std_normalization=True
                                                ) 

In [ ]:
def load_data(data_type, target_size, batch_size):
    # data path
    train_data_path = r'D:\项目资料\基因表达\数据'
    val_data_path = r'D:\项目资料\基因表达\数据'
    test_data_path = r'D:\项目资料\基因表达\数据'
    
    train_gen = train_img_gen.flow_from_directory(train_data_path,
                                    target_size = target_size,
                                    color_mode = 'rgb',
#                                     classes = [''],
                                    class_mode = 'categorical',
                                    batch_size = batch_size,
                                    shuffle = True,
                                    seed = None,
                                    interpolation = 'nearest',
                                    keep_aspect_ratio=False
                                                 )
    print('Train data loaded: total: %s, batch size: %s, num classes: %s\n' % (train_gen.n, train_gen.batch_size, train_gen.num_classes))
    
    val_gen = val_img_gen.flow_from_directory(val_data_path,
                                    target_size = target_size,
                                    color_mode = 'rgb',
#                                     classes = [''],
                                    class_mode = 'categorical',
                                    batch_size = batch_size,
                                    shuffle = False,
                                    seed = None,
                                    interpolation = 'nearest',
                                    keep_aspect_ratio=False
                                             )
    print('Val data loaded: total: %s, batch size: %s, num classes: %s\n' % (val_gen.n, val_gen.batch_size, val_gen.num_classes))
    
    return train_gen, val_gen

In [ ]:
def build_model(type='Mobilenet', num_classes=27):
#     scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)

    base_model, input_shape = get_base_model(type), get_basemodel_shape(type)
    base_model.trainable = False
    inputs = keras.Input(shape=input_shape)
    
    # 模型定义
#     x = scale_layer(inputs)
    x = inputs
    x = base_model(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)  # Regularize with dropout
    x = tf.keras.layers.Dense(num_classes)(x)
#     outputs = tf.keras.layers.Softmax()(x)
    outputs = x
    model = tf.keras.Model(inputs, outputs)
    
    model.summary()
    
    return model

In [ ]:
def scheduler(epoch, lr):
    if epoch > 2:
        lr = lr * tf.math.exp(-0.051)
    return lr
    
early_stop = EarlyStopping(monitor='val_accuracy', patience=3)
scheduler_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

def get_callbacks(modl_type, data_type, unfrozen_layers=None):
    model_checkpoint = ModelCheckpoint(get_mdl_name(data_type, modl_type, unfrozen_layers), monitor='val_accuracy', verbose=2, save_best_only=True)
    callbacks = [early_stop, model_checkpoint, scheduler_callback]
    return callbacks

In [ ]:
scheduler(21, 0.01)

In [ ]:
modl_types = ['VGG16', 'ResNet50', 'EfficientNet-B4']
data_types = ['esophagus_28', 'pharynx_5']
batch_sizes = 4, 32
def transfer_learning(mdl_idx, data_idx, init_lr=1e-2, epochs=5, batch_size=64):
    modl_type = modl_types[mdl_idx]
    img_shape = get_basemodel_shape(modl_type)[:2]
    data_type = data_types[data_idx]
    BATCH_SIZE = batch_size
    
    # 参数
    num_clses = int(data_type.split('_')[1])

    # 定义模型
    model = build_model(type=modl_type, num_classes=num_clses)
    model.compile(optimizer = Adam(learning_rate = init_lr), 
                      loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
                      metrics = ['accuracy'])

    # 加载数据
    train_gen, val_gen = load_data(data_type, img_shape, BATCH_SIZE)


    # 训练
    history = model.fit(train_gen, steps_per_epoch=train_gen.n // BATCH_SIZE, epochs=epochs,
                        validation_data=val_gen, validation_steps=val_gen.n // BATCH_SIZE,
                        callbacks=get_callbacks(modl_type, data_type), workers=1)


In [ ]:
def finetune(mdl_idx, data_idx, chpkt_path, init_lr=1e-5, unfrozen_layers=-200, epochs=5, batch_size=16):
    modl_type = modl_types[mdl_idx]
    img_shape = get_basemodel_shape(modl_type)[:2]
    data_type = data_types[data_idx]
    BATCH_SIZE = batch_size
    
    # 参数
    num_clses = int(data_type.split('_')[1])
    
    # 定义模型
    model = tf.keras.models.load_model(chpkt_path)
#     model.trainable = True
    print(len(model.layers[1].layers))
    for layer in model.layers[1].layers[unfrozen_layers:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True
            
    model.compile(optimizer = Adam(learning_rate = init_lr), 
                          loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
                          metrics = ['accuracy'])
    model.summary()
    
    # 加载数据
    train_gen, val_gen = load_data(data_type, img_shape, BATCH_SIZE)


    # 训练
    history = model.fit(train_gen, steps_per_epoch=train_gen.n // BATCH_SIZE, epochs=epochs,
                        validation_data=val_gen, validation_steps=val_gen.n // BATCH_SIZE,
                        callbacks=get_callbacks(modl_type, data_type, unfrozen_layers), workers=1)

In [ ]:
mdl_idx, data_idx = 2, 0
# transfer_learning(mdl_idx, data_idx, 1e-2, 3, 16)
# finetune(mdl_idx, data_idx, find_best_chkpt(mdl_idx, data_idx), 1e-5, 120, 50, 4)
finetune(mdl_idx, data_idx, find_best_chkpt(mdl_idx, data_idx), 1e-6, 120, 20, 4)